# Travel Cost Analysis for Discontinue Silver Line Library Segment Service Scenario
### This analysis consists of two parts: 
1. Total additional travel cost added from the lost ridership. 
2. Travel cost increase for commuting riders. 

In [49]:
import pandas as pd 
import numpy as np 

In [50]:
df = pd.read_excel('PRT_station_distance_updated.xlsx')
df.head()

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,FID_1,stop_id,stop_code,stop_name,stop_lat,stop_lon,...,transportation_X6059_X,transportation_X6060_X,transportation_X6061_X,transportation_X6062_X,transportation_X6063_X,Shape_Length,Shape_Area,NEAR_FID,Dist_Downtown,Dis_to_Washington_Jxn
0,1,1,1,21,5073,X14476,99935,SOUTH HILLS JUNCTION STATION,40.420810,-80.006928,...,1949,125,743,1049,32,1869.324997,159367.861801,1,1.275158,4.841573
1,2,1,3,16,3350,P70025,99991,GATEWAY STATION,40.441705,-80.004104,...,0,0,0,0,0,674.927374,28809.544908,1,0.416277,6.285285
2,3,1,3,17,3367,P70021,99992,WOOD STREET STATION,40.442338,-79.999525,...,0,0,0,0,0,674.927374,28809.544908,1,0.416277,6.285285
3,4,1,5,15,1564,N71210,99999,NORTH SIDE STATION,40.448092,-80.010515,...,0,0,0,0,0,1820.897991,151792.908828,1,0.890242,6.532254
4,5,1,8,12,1350,X70001,99918,BOGGS STATION,40.416702,-80.010290,...,0,0,0,0,0,123.812403,589.499413,1,1.902860,4.331867


In [51]:
df.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'JOIN_FID', 'FID_1', 'stop_id',
       'stop_code', 'stop_name', 'stop_lat', 'stop_lon',
       ...
       'transportation_X6059_X', 'transportation_X6060_X',
       'transportation_X6061_X', 'transportation_X6062_X',
       'transportation_X6063_X', 'Shape_Length', 'Shape_Area', 'NEAR_FID',
       'Dist_Downtown', 'Dis_to_Washington_Jxn'],
      dtype='object', length=104)

In [52]:
df[df['routes'] == 'SLVR'].stop_name.unique()

array(['SANDY CREEK', 'BEAGLE', 'LIBRARY STATION', 'HILLCREST',
       'SOUTH PARK ROAD', 'MESTA', 'MUNROE', 'LYTLE', 'KINGS SCHOOL ROAD',
       'SARAH', 'WEST LIBRARY', 'LOGAN ROAD'], dtype=object)

In [63]:
# Get the Library segment of the Silver Line
df_slvr_section = df[df['routes'] == 'SLVR']

### Section 1: Total additional travel cost added from the lost ridership

#### Calculate the average distance of riders who onboard at each station to Washington Junction and Downtown. This is calculated by taking the average of distances of all block groups within 0.25 miles radius of each station to Washington Jxn and Downtown. 

In [62]:
station_distance = df_slvr_section.groupby(['stop_name', 'fy24_av_on'])[['Dis_to_Washington_Jxn', 'Dist_Downtown']].mean().reset_index()
station_distance

,stop_name,fy24_av_on,Dis_to_Washington_Jxn,Dist_Downtown
0,BEAGLE,6,3.472539,9.609326
1,HILLCREST,4,0.459757,6.630640
2,KINGS SCHOOL ROAD,6,2.949032,9.102070
3,LIBRARY STATION,14,4.531391,10.567640
4,LOGAN ROAD,6,2.791382,8.949656
5,LYTLE,12,1.276811,7.453103
6,MESTA,11,1.477176,7.644447
7,MUNROE,6,2.020968,8.166384
8,SANDY CREEK,8,3.833473,9.954411
9,SARAH,4,2.433964,8.584222


*Caculate total miles travel of the existing riders of the Silver Line if they can only onboard on Washington Junction station*

In [65]:
total_miles_travel_washingtonjxn = (station_distance['fy24_av_on'] * station_distance['Dis_to_Washington_Jxn']).sum()
total_miles_travel_washingtonjxn

343.1954757180461

Assume a average MPG of 33 miles (2022 Hyundai Tuscon); and a current national fuel price of $3.397; source: https://www.thezebra.com/resources/driving/fuel-cost-calculator/

In [66]:
# Calculate cost of gas 
total_gas_cost = total_miles_travel_washingtonjxn / 33 * 3.397
total_gas_cost

35.32833427315765

In [67]:
# Calculate cost of time
avg_speed = 50
travel_time = total_miles_travel_washingtonjxn / avg_speed
total_travel_cost = travel_time * 23 # Assume hourly value of time is $23/hr --- average income of middle income family in PA
total_travel_cost

157.8699188303012

In [68]:
# Calculate yearly values
total_gas_cost_year = total_gas_cost * 365
total_travel_cost_year = total_travel_cost * 365
total_cost_year_min = total_gas_cost_year + total_travel_cost_year
total_cost_year_min

70517.36238276248

*Conduct the same analysis but assume existing riders need to drive to Pittsburgh Downtown* 

In [69]:
total_miles_travel_downtown = (station_distance['fy24_av_on'] * station_distance['Dist_Downtown']).sum()
total_miles_travel_downtown

1054.3255406082885

In [70]:
# Calculate cost of gas 
total_gas_cost = total_miles_travel_downtown / 33 * 3.397
total_gas_cost

108.53163216504109

In [71]:
# Calculate cost of time
avg_speed = 50
travel_time = total_miles_travel_downtown / avg_speed
total_travel_cost = travel_time * 23 # Assume hourly value of time is $23/hr --- average income of middle income family in PA
total_travel_cost

484.9897486798127

In [72]:
# Calculate yearly values
total_gas_cost_year = total_gas_cost * 365
total_travel_cost_year = total_travel_cost * 365
total_cost_year_max = total_gas_cost_year + total_travel_cost_year
total_cost_year_max

216635.30400837163

This means that given the existing daily ridership in the Library section of Silver Line, if the existing riders have to drive to Washington Junction to take the Blue Line, this will incur a daily additional miles travel of 343 miles, which is estimated to incur an annual cost from gas and time of $70,000; if the existing riders have to drive to Pittsburgh Downtown, this will incur a daily additional miles of travel of 1054 miles, which is estimated to incur an annual cost from gas and time of $0.2 million. <br>
##### In conclusion, based on existing daily ridership in the Library segment of the Silver line, discontinuing service is estimated to incur an annual cost from gas and time between $70,000 and $200,000.

### Section 2: Travel cost increase for commuting riders

In [73]:
df_slvr_section.head()

,OBJECTID,Join_Count,TARGET_FID,JOIN_FID,FID_1,stop_id,stop_code,stop_name,stop_lat,stop_lon,...,transportation_X6059_X,transportation_X6060_X,transportation_X6061_X,transportation_X6062_X,transportation_X6063_X,Shape_Length,Shape_Area,NEAR_FID,Dist_Downtown,Dis_to_Washington_Jxn
19,20,1,27,26,5357,X14415,99945,SANDY CREEK,40.296477,-80.030387,...,2100,118,924,998,60,1800.245535,188615.997243,1,9.794397,3.669510
20,21,1,27,30,6373,X13810,99982,BEAGLE,40.300909,-80.031477,...,2100,118,924,998,60,1800.245535,188615.997243,1,9.794397,3.669510
25,26,1,39,20,5057,X14447,99942,LIBRARY STATION,40.287073,-80.020933,...,877,50,376,435,17,1047.200853,53847.860324,1,10.659559,4.588192
27,28,1,46,4,1009,X14125,99964,HILLCREST,40.345592,-80.028853,...,2727,130,1216,1314,67,5189.713338,659924.114758,1,6.300756,0.106842
36,37,1,62,25,5180,X14480,99933,SOUTH PARK ROAD,40.327821,-80.030536,...,2171,103,968,1046,53,2305.183825,287209.307543,1,7.814661,1.640925


To calculate the average number of commuting riders at each station, I will use groupby and sum up the total number of commuters in the 2-miles radius of each station. 

In [74]:
commuter_count = df_slvr_section.groupby(by = 'stop_name', as_index=True)[['commute_ACSSTRTCAR', 'commute_ACSSUBWAY']].sum().reset_index()
commuter_count

,stop_name,commute_ACSSTRTCAR,commute_ACSSUBWAY
0,BEAGLE,17,16
1,HILLCREST,46,17
2,KINGS SCHOOL ROAD,31,15
3,LIBRARY STATION,25,0
4,LOGAN ROAD,16,11
5,LYTLE,9,38
6,MESTA,2,29
7,MUNROE,5,25
8,SANDY CREEK,7,10
9,SARAH,16,14


In [75]:
commuter_count['total_strtcar_subway'] = commuter_count['commute_ACSSTRTCAR']+ commuter_count['commute_ACSSUBWAY']
commuter_count

,stop_name,commute_ACSSTRTCAR,commute_ACSSUBWAY,total_strtcar_subway
0,BEAGLE,17,16,33
1,HILLCREST,46,17,63
2,KINGS SCHOOL ROAD,31,15,46
3,LIBRARY STATION,25,0,25
4,LOGAN ROAD,16,11,27
5,LYTLE,9,38,47
6,MESTA,2,29,31
7,MUNROE,5,25,30
8,SANDY CREEK,7,10,17
9,SARAH,16,14,30


Small data integrity check--Make sure each station is mapped to a census block group once.

In [39]:
df_slvr_section[df_slvr_section['stop_name']=='WEST LIBRARY'][['TFID']].duplicated().any()

False

In [78]:
# Merge commuter_count and station_distance tables
merged = commuter_count.merge(right = station_distance, on = 'stop_name', how = 'outer')
merged

,stop_name,commute_ACSSTRTCAR,commute_ACSSUBWAY,total_strtcar_subway,fy24_av_on,Dis_to_Washington_Jxn,Dist_Downtown
0,BEAGLE,17,16,33,6,3.472539,9.609326
1,HILLCREST,46,17,63,4,0.459757,6.630640
2,KINGS SCHOOL ROAD,31,15,46,6,2.949032,9.102070
3,LIBRARY STATION,25,0,25,14,4.531391,10.567640
4,LOGAN ROAD,16,11,27,6,2.791382,8.949656
5,LYTLE,9,38,47,12,1.276811,7.453103
6,MESTA,2,29,31,11,1.477176,7.644447
7,MUNROE,5,25,30,6,2.020968,8.166384
8,SANDY CREEK,7,10,17,8,3.833473,9.954411
9,SARAH,16,14,30,4,2.433964,8.584222


In [79]:
# Total commuters impacted
merged['total_strtcar_subway'].sum()

407

Calculate additional cost for those commuters; Assume they all have to go to Washington Junction to take Blue Line

In [80]:
merged['daily_dist_washington'] = merged['Dis_to_Washington_Jxn'] * 2

In [ ]:
# Calculate cost of gas (range)
daily_gas_cost_max = merged['daily_dist_washington'].max() / 33 * 3.397
daily_gas_cost_min = merged['daily_dist_washington'].min() / 33 * 3.397

print('The range of daily gas cost from the travel is between: ', daily_gas_cost_min, ' and ', daily_gas_cost_max)

The range of daily gas cost from the travel is between:  0.09465428411386079  and  0.9329172910981819


In [82]:
# Calculate cost of time
avg_speed = 50
daily_travel_time_max = merged['daily_dist_washington'].max() / avg_speed
daily_travel_time_min = merged['daily_dist_washington'].min() / avg_speed

daily_travel_cost_max = daily_travel_time_max * 23 # Assume hourly value of time is $23/hr --- average income of middle income family in PA
daily_travel_cost_min = daily_travel_time_min * 23

print('The range of daily cost of time from the travel is between: ', daily_travel_time_min, ' and ', daily_travel_time_max)

The range of daily cost of time from the travel is between:  0.0183902936459076  and  0.1812556408963203


In [84]:
# Calculate total cost (if drive)
daily_cost_drive_min = daily_gas_cost_min + daily_travel_cost_min
daily_cost_drive_max = daily_gas_cost_max + daily_travel_cost_max
# Assume 21 working days in a month
monthly_cost_drive_min = daily_cost_drive_min * 21
monthly_cost_drive_max = daily_cost_drive_max * 21
print('The range of monthly cost of travel if commuters have to drive to Washington Junction Station is between: ', monthly_cost_drive_min, ' and ', monthly_cost_drive_max)


The range of monthly cost of travel if commuters have to drive to Washington Junction Station is between:  10.870251797364446  and  107.13773766598452


In [85]:
merged['daily_dist_washington'].max()

9.062782044816014

In [87]:
merged['daily_dist_washington'].min()

0.91951468229538

Using ChatGPT, the range of travel distance, if traveled by UBer, is estimated to cost between $7~$27 per day, and therefore $147~$567 per month. 